# Heat Mapping <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/29_heat_mapping.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before using the graph widget, install all necessary packages and initialize your widget.

In [1]:
%pip install yfiles_jupyter_graphs --quiet
from yfiles_jupyter_graphs import GraphWidget
w = GraphWidget()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.0 MB/s eta 0:00:00


You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [2]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/29_heat_mapping.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mapping Function

The heatmapping is slightly different than other mappings, as there is one combined mapping for edges and nodes

There are get, set and delete methods for the heatmap.
- you can set a new heat mapping with ```w.set_heat_mapping```
- you can get the current heat mapping with ```w.get_heat_mapping```
- you can delete a custom heat mapping with ```w.del_heat_mapping```

If no custom mapping is set the default mapping is used.

In [3]:
print(w.default_heat_mapping.__doc__)

The default scale factor mapping for nodes.

                Provides constant value of None for all elements.

                Parameters
                ----------
                index: int (optional)
                element: typing.Dict

                Notes
                -----
                This is the default value for the `heat_mapping` property.
                Can be 'overwritten' by setting the property
                with a function of the same signature.

                If the given mapping function has only one parameter (that is not typed as int),
                then it will be called with the element (typing.Dict) as first parameter.

                Example
                -------
                .. code::

                   from yfiles_jupyter_graphs import GraphWidget
                   w = GraphWidget()
                   def custom_heat_mapping(element: typing.Dict):
                   ...
                   w.set_heat_mapping(custom_heat_mapping)

        

In [4]:
def heatmap(element):
    load = 0.5
    if 'start' in element:
        #edge case
        return 0
    else:
        # Calculate load based on number of edges
        load += sum(0.1 for edge in w.edges if edge['end'] == element['id'])
        load += sum(-0.1 for edge in w.edges if edge['start'] == element['id'])
        # Calculate load based on capacity (if available)
        if 'properties' in element and 'capacity' in element['properties']:
            load += 1 - (element['properties']['capacity'] / 100)
        # Calculate load based on duration (if available)
        if 'properties' in element and 'duration' in element['properties']:
            load += element['properties']['duration'] / 10
        load = min(1, max(0, load))
        return load


We use a graph representing a production chain and add our heatmap according to the expected step load:

In [5]:
w.set_heat_mapping(heatmap)
w.nodes = [
    { "id": 1, "properties": {"label": "Start", "capacity": 50 }},
    { "id": 3, "properties": {"label": "Evaluation", "capacity": 20 }},
    { "id": 4, "properties": {"label": "Move to Backlog"}},
    { "id": 5, "properties": {"label": "Prepare", "capacity": 20 }},
    { "id": 6, "properties": {"label": "Prepare Shortcut"}},
    { "id": 7, "properties": {"label": "Step A-1"}},
    { "id": 9, "properties": {"label": "Advance", "duration": 3 }},
    { "id": 10, "properties": {"label": "Early Discard"}},
    { "id": 11, "properties": {"label": "Step A-2"}},
    { "id": 12, "properties": {"label": "Quick Preparation"}},
    { "id": 13, "properties": {"label": "Backlog", "capacity": 100 }},
    { "id": 14, "properties": {"label": "Step B"}},
    { "id": 16, "properties": {"label": "End of Preparation"}},
    { "id": 17, "properties": {"label": "Buffer", "capacity": 30, "duration": 10 }},
    { "id": 18, "properties": {"label": "Main Processing", "capacity": 70, "duration": 2 }},
    { "id": 19, "properties": {"label": "Refinement 1", "capacity": 20, "duration": 1 }},
    { "id": 20, "properties": {"label": "Refinement 2", "capacity": 60, "duration": 4 }},
    { "id": 21, "properties": {"label": "Testing", "capacity": 70, "duration": 1 }},
    { "id": 22, "properties": {"label": "Delivery", "capacity": 20 }},
    { "id": 23, "properties": {"label": "Rejection", "capacity": 100 }},
    { "id": 24, "properties": {"label": "Store", "capacity": 50, "duration": 5 }}
]

w.edges = [
    { "start": 1, "end": 5, "properties": {} },
    { "start": 1, "end": 2, "properties": {} },
    { "start": 1, "end": 3 , "properties": {} },
    { "start": 1, "end": 4, "properties": { "probability": 0.1 }},
    { "start": 16, "end": 17, "properties": {}  },
    { "start": 16, "end": 18, "properties": {}  },
    { "start": 18, "end": 19, "properties": {}  },
    { "start": 5, "end": 11, "properties": {}  },
    { "start": 9, "end": 6, "properties": { "probability": 0.1 }},
    { "start": 5, "end": 7, "properties": { "probability": 3 }},
    { "start": 2, "end": 8 , "properties": {} },
    { "start": 6, "end": 12 , "properties": {} },
    { "start": 3, "end": 9 , "properties": {} },
    { "start": 3, "end": 10, "properties": { "probability": 0.1 }},
    { "start": 4, "end": 13 , "properties": {} },
    { "start": 11, "end": 14 , "properties": {} },
    { "start": 7, "end": 14, "properties": {}  },
    { "start": 5, "end": 16, "properties": { "probability": 0.1 }},
    { "start": 14, "end": 16 , "properties": {} },
    { "start": 18, "end": 21 , "properties": {} },
    { "start": 21, "end": 23, "properties": { "probability": 0.1 }},
    { "start": 21, "end": 24 , "properties": {} },
    { "start": 21, "end": 22 , "properties": {} },
    { "start": 13, "end": 15 , "properties": {} },
    { "start": 12, "end": 17 , "properties": {} },
    { "start": 18, "end": 22 , "properties": {} },
    { "start": 10, "end": 15 , "properties": {} },
    { "start": 9, "end": 20, "properties": {}  },
    { "start": 19, "end": 21 , "properties": {} },
    { "start": 17, "end": 18 , "properties": {} },
    { "start": 20, "end": 21 , "properties": {} }
]
w.hierarchic_layout()
w.get_heat_mapping()

<function __main__.heatmap(element)>

In [6]:
display(w)

GraphWidget(layout=Layout(height='710px', width='100%'))

If the heat mapping is deleted, the heatmap mapping reverts back to the default mapping.

In [7]:
w.del_heat_mapping()
w.get_heat_mapping()

<function yfiles_jupyter_graphs.widget.GraphWidget.default_heat_mapping(element: Dict)>